# RNU4-2
[ReNU syndrome (RENU)](https://omim.org/entry/620851) is caused by heterozygous mutation in the RNU4-2 gene.

RNU4-2 is one of two U4 snRNA genes.

[Chen Y, et al. (2024)](https://pubmed.ncbi.nlm.nih.gov/38991538/) report:
> The n.64_65insT variant is one of six single base insertions that we observe in the 18 bp critical region in individuals with NDD, in a total of 100 individuals across cohorts. By contrast, single base insertions are very rare in population cohorts. Although we do also observe some SNVs in this region in individuals with NDD, our initial data suggest these SNVs may result in a milder phenotype. However, given this observation is based on only four fully phenotyped individuals, it needs to be confirmed in larger cohorts. 

In [16]:
import hpotk
import gpsea

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2024-08-13')
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-08-13
Using gpsea version 0.5.1.dev0


# RNU4-2
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript for RNU4-2. RNU4-2 is non-coding and thus there is no protein identifier.

In [17]:
gene_symbol = 'RNU4-2'
mane_tx_id = 'NR_003137.3'

In [18]:
from ppktstore.registry import configure_phenopacket_registry

phenopacket_store_release = '0.1.21'  # Update, if necessary
registry = configure_phenopacket_registry()

#with registry.open_phenopacket_store(release=phenopacket_store_release) as ps:
#    phenopackets = tuple(ps.iter_cohort_phenopackets(gene_symbol))

#print(f'Loaded {len(phenopackets)} phenopackets')

from gpsea.preprocessing import configure_caching_cohort_creator

cohort_name = "RNU4-2"	


cohort_creator = configure_caching_cohort_creator(hpo)
from gpsea.preprocessing import load_phenopacket_folder
pp_dir = '/Users/robin/GIT/phenopacket-store/notebooks/RNU4-2/phenopackets/'
cohort, qc_results = load_phenopacket_folder(pp_dir, cohort_creator)  
qc_results.summarize()


Individuals Processed: 50individuals [00:03, 15.93individuals/s]
Validated under permissive policy
Showing errors and warnings
Phenopackets
  patient #12
    phenotype-features
     errors:



In [19]:
from gpsea.view import CohortViewable
viewer = CohortViewable(hpo)
viewer.process(cohort=cohort, transcript_id=mane_tx_id)

HPO Term,ID,Seen in n individuals
Delayed ability to walk,HP:0031936,46
Intellectual disability,HP:0001249,40
Absent speech,HP:0001344,40
Seizure,HP:0001250,39
Hypotonia,HP:0001252,39
Short stature,HP:0004322,38
Severe global developmental delay,HP:0011344,35
Failure to thrive,HP:0001508,30
Constipation,HP:0002019,30
Gastroesophageal reflux,HP:0002020,24


In [20]:
from gpsea.analysis.predicate.phenotype import prepare_predicates_for_terms_of_interest
from gpsea.analysis.pcats.stats import FisherExactTest
from gpsea.analysis.mtc_filter import HpoMtcFilter
from gpsea.analysis.pcats import HpoTermAnalysis

pheno_predicates = prepare_predicates_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)

mtc_filter = HpoMtcFilter.default_filter(hpo=hpo, term_frequency_threshold=0.4, annotation_frequency_threshold=0.4)
mtc_correction = 'fdr_bh'
statistic = FisherExactTest()

analysis = HpoTermAnalysis(
    count_statistic=statistic,
    mtc_filter=mtc_filter,
    mtc_correction=mtc_correction,
    mtc_alpha=0.05,
)

In [25]:
from gpsea.analysis.predicate.genotype import VariantPredicates, monoallelic_predicate
from gpsea.model import VariantEffect


is_n64_65insT = VariantPredicates.variant_key("12_120291839_120291839_T_TA") # n.64_65insT
is_b = VariantPredicates.variant_key("12_120291835_120291835_G_GT") #12_120291835_120291835_G_GT
is_c = VariantPredicates.variant_key("12_120291839_120291839_T_TC") # 12_120291839_120291839_T_TC
is_d = VariantPredicates.variant_key("12_120291826_120291826_T_TA") # 12_120291826_120291826_T_TA

is_insertion = is_n64_65insT | is_b | is_c | is_d 


is_n64_65insT_predicate = monoallelic_predicate(
    a_predicate=is_n64_65insT,
    b_predicate=~is_n64_65insT,
    names=("n.64_65insT", "other")
)

is_insertion_predicate = monoallelic_predicate(
    a_predicate=is_insertion,
    b_predicate=~is_insertion,
    names=("insertion", "other")
)

is_n64_65insT_predicate.display_question()

'Allele group: n.64_65insT, other'

In [23]:
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=is_n64_65insT_predicate,
    pheno_predicates=pheno_predicates,
)
from gpsea.view import MtcStatsViewer

viewer = MtcStatsViewer()
viewer.process(result)

Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.4,131
HMF03,Skipping term because of a child term with the same individual counts,12
HMF08,Skipping general term,59
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.4,103


In [24]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=result)

Allele group                                       n.64_65insT         other  \
                                                         Count Percent Count   
Severe global developmental delay [HP:0011344]           33/38     87%   2/7   
Moderate global developmental delay [HP:0011343]          5/38     13%   5/7   
Absent speech [HP:0001344]                               37/41     90%   3/7   
Constipation [HP:0002019]                                28/38     74%   2/6   
Hearing impairment [HP:0000365]                           6/38     16%   3/7   
Delayed ability to walk [HP:0031936]                     40/40    100%   6/7   
Primary microcephaly [HP:0011451]                        19/34     56%   1/5   
Secondary microcephaly [HP:0005484]                       6/34     18%   2/5   
Seizure [HP:0001250]                                     32/42     76%   7/7   
Autistic behavior [HP:0000729]                           21/39     54%   1/5   
Failure to thrive [HP:0001508]                           27/37     73%   3/6   
Strabismus [HP:0000486]                                  21/39     54%   2/6   
Nystagmus [HP:0000639]                                   16/34     47%   2/6   
Growth delay [HP:0001510]                                33/33    100%   6/6   
Delayed gross motor development [HP:0002194]             40/40    100%   6/6   
Neurodevelopmental delay [HP:0012758]                    43/43    100%   7/7   
Neurodevelopmental abnormality [HP:0012759]              43/43    100%   7/7   
Feeding difficulties [HP:0011968]                        29/31     94%   4/4   
Abdominal symptom [HP:0011458]                           34/34    100%   4/4   
Short stature [HP:0004322]                               33/43     77%   5/7   
Global developmental delay [HP:0001263]                  43/43    100%   7/7   
Intellectual disability [HP:0001249]                     39/39    100%   6/6   
Aplasia/Hypoplasia involving the central nervou...       36/36    100%   5/5   
Morphological central nervous system abnormalit...       40/40    100%   6/6   
Decreased body weight [HP:0004325]                       27/27    100%   3/3   
Microcephaly [HP:0000252]                                32/32    100%   5/5   
Abnormal axial skeleton morphology [HP:0009121]          35/35    100%   5/5   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]          34/34    100%   5/5   
Abnormal cerebral morphology [HP:0002060]                37/37    100%   6/6   
Abnormal forebrain morphology [HP:0100547]               37/37    100%   6/6   
Abnormal brain morphology [HP:0012443]                   40/40    100%   6/6   
Abnormal speech pattern [HP:0002167]                     42/42    100%   5/5   
Abnormal communication [HP:0034434]                      42/42    100%   5/5   
Delayed speech and language development [HP:000...       42/42    100%   5/5   
Abnormal cerebral white matter morphology [HP:0...       19/19    100%   1/1   
Abnormality of the mouth [HP:0000153]                    33/33    100%   5/5   
Abnormal oral cavity morphology [HP:0000163]             33/33    100%   5/5   
Abnormal ocular adnexa morphology [HP:0030669]           18/18    100%   2/2   
Hypotonia [HP:0001252]                                   36/36    100%   4/4   
Abnormal muscle tone [HP:0003808]                        37/37    100%   4/4   
Abnormal lip morphology [HP:0000159]                     23/23    100%   3/3   
Abnormal nasal morphology [HP:0005105]                   19/19    100%   2/2   
Abnormal pinna morphology [HP:0000377]                   20/20    100%   2/2   
Abnormal conjugate eye movement [HP:0000549]             21/21    100%   2/2   
Gastroesophageal reflux [HP:0002020]                     21/38     55%   3/6   
Abnormal esophagus physiology [HP:0025270]               21/21    100%   3/3   
Diagnostic behavioral phenotype [HP:0025783]             21/21    100%   1/1   
Atypical behavior [HP:0000708]                           24/24    100%   2/2   


In [26]:
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=is_insertion_predicate,
    pheno_predicates=pheno_predicates,
)
summarize_hpo_analysis(hpo=hpo, result=result)

Allele group                                       insertion         other  \
                                                       Count Percent Count   
Moderate global developmental delay [HP:0011343]        6/41     15%   4/4   
Severe global developmental delay [HP:0011344]         35/41     85%   0/4   
Absent speech [HP:0001344]                             39/44     89%   1/4   
Failure to thrive [HP:0001508]                         30/40     75%   0/3   
Constipation [HP:0002019]                              30/41     73%   0/3   
Delayed ability to walk [HP:0031936]                   43/43    100%   3/4   
Intrauterine growth retardation [HP:0001511]            6/41     15%   2/4   
Nystagmus [HP:0000639]                                 18/37     49%   0/3   
Short stature [HP:0004322]                             36/46     78%   2/4   
Seizure [HP:0001250]                                   35/45     78%   4/4   
Primary microcephaly [HP:0011451]                      19/36     53%   1/3   
Strabismus [HP:0000486]                                22/42     52%   1/3   
Growth delay [HP:0001510]                              36/36    100%   3/3   
Delayed gross motor development [HP:0002194]           43/43    100%   3/3   
Neurodevelopmental delay [HP:0012758]                  46/46    100%   4/4   
Neurodevelopmental abnormality [HP:0012759]            46/46    100%   4/4   
Feeding difficulties [HP:0011968]                      32/34     94%   1/1   
Abdominal symptom [HP:0011458]                         37/37    100%   1/1   
Global developmental delay [HP:0001263]                46/46    100%   4/4   
Intellectual disability [HP:0001249]                   42/42    100%   3/3   
Aplasia/Hypoplasia involving the central nervou...     38/38    100%   3/3   
Morphological central nervous system abnormalit...     43/43    100%   3/3   
Microcephaly [HP:0000252]                              34/34    100%   3/3   
Abnormal axial skeleton morphology [HP:0009121]        37/37    100%   3/3   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]        36/36    100%   3/3   
Abnormal cerebral morphology [HP:0002060]              40/40    100%   3/3   
Abnormal forebrain morphology [HP:0100547]             40/40    100%   3/3   
Abnormal brain morphology [HP:0012443]                 43/43    100%   3/3   
Abnormal speech pattern [HP:0002167]                   44/44    100%   3/3   
Abnormal communication [HP:0034434]                    44/44    100%   3/3   
Delayed speech and language development [HP:000...     44/44    100%   3/3   
Abnormality of the mouth [HP:0000153]                  36/36    100%   2/2   
Abnormal oral cavity morphology [HP:0000163]           36/36    100%   2/2   
Abnormal ocular adnexa morphology [HP:0030669]         18/18    100%   2/2   
Hypotonia [HP:0001252]                                 38/38    100%   2/2   
Abnormal muscle tone [HP:0003808]                      39/39    100%   2/2   
Abnormal nasal morphology [HP:0005105]                 20/20    100%   1/1   
Abnormal pinna morphology [HP:0000377]                 21/21    100%   1/1   
Abnormal conjugate eye movement [HP:0000549]           22/22    100%   1/1   
Gastroesophageal reflux [HP:0002020]                   22/41     54%   2/3   
Abnormal esophagus physiology [HP:0025270]             22/22    100%   2/2   
Autistic behavior [HP:0000729]                         21/41     51%   1/3   
Diagnostic behavioral phenotype [HP:0025783]           21/21    100%   1/1   
Atypical behavior [HP:0000708]                         25/25    100%   1/1   

Allele group                                                                   \
                                                   Percent Corrected p values   
Moderate global developmental delay [HP:0011343]      100%           0.031008   
Severe global developmental delay [HP:0011344]          0%           0.031008   
Absent speech [HP:0001344]                             25%           0.174119   
Failur

In [27]:
from gpsea.analysis.predicate.genotype import sex_predicate
result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_predicate=sex_predicate(),
    pheno_predicates=pheno_predicates,
)
from gpsea.view import summarize_hpo_analysis
summary_df = summarize_hpo_analysis(hpo, result)
summary_df

Sex of the individual                              FEMALE           MALE  \
                                                    Count Percent  Count   
Hypothyroidism [HP:0000821]                          0/11      0%  10/22   
Feeding difficulties [HP:0011968]                   13/15     87%  20/20   
Gastroesophageal reflux [HP:0002020]                 8/18     44%  16/26   
Short stature [HP:0004322]                          18/22     82%  20/28   
Nystagmus [HP:0000639]                               6/16     38%  12/24   
Seizure [HP:0001250]                                16/21     76%  23/28   
Severe global developmental delay [HP:0011344]      15/20     75%  20/25   
Failure to thrive [HP:0001508]                      11/17     65%  19/26   
Primary microcephaly [HP:0011451]                    9/16     56%  11/23   
Constipation [HP:0002019]                           11/17     65%  19/27   
Growth delay [HP:0001510]                           18/18    100%  21/21   
Delayed ability to walk [HP:0031936]                19/19    100%  27/28   
Delayed gross motor development [HP:0002194]        19/19    100%  27/27   
Neurodevelopmental delay [HP:0012758]               22/22    100%  28/28   
Neurodevelopmental abnormality [HP:0012759]         22/22    100%  28/28   
Abdominal symptom [HP:0011458]                      15/15    100%  23/23   
Global developmental delay [HP:0001263]             22/22    100%  28/28   
Intellectual disability [HP:0001249]                18/18    100%  27/27   
Aplasia/Hypoplasia involving the central nervou...  17/17    100%  24/24   
Morphological central nervous system abnormalit...  20/20    100%  26/26   
Decreased body weight [HP:0004325]                  11/11    100%  19/19   
Microcephaly [HP:0000252]                           15/15    100%  22/22   
Abnormal axial skeleton morphology [HP:0009121]     17/17    100%  23/23   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]     17/17    100%  22/22   
Abnormal cerebral morphology [HP:0002060]           20/20    100%  23/23   
Abnormal forebrain morphology [HP:0100547]          20/20    100%  23/23   
Abnormal brain morphology [HP:0012443]              20/20    100%  26/26   
Absent speech [HP:0001344]                          18/21     86%  22/27   
Abnormal speech pattern [HP:0002167]                21/21    100%  26/26   
Abnormal communication [HP:0034434]                 21/21    100%  26/26   
Delayed speech and language development [HP:000...  21/21    100%  26/26   
Abnormal cerebral white matter morphology [HP:0...  11/11    100%    9/9   
Abnormality of the mouth [HP:0000153]               16/16    100%  22/22   
Abnormal oral cavity morphology [HP:0000163]        16/16    100%  22/22   
Abnormal ocular adnexa morphology [HP:0030669]        9/9    100%  11/11   
Hypotonia [HP:0001252]                              18/18    100%  22/22   
Abnormal muscle tone [HP:0003808]                   19/19    100%  22/22   
Abnormal lip morphology [HP:0000159]                11/11    100%  15/15   
Abnormal nasal morphology [HP:0005105]                7/7    100%  14/14   
Abnormal pinna morphology [HP:0000377]                7/7    100%  15/15   
Strabismus [HP:0000486]                              9/17     53%  14/28   
Abnormal conjugate eye movement [HP:0000549]          9/9    100%  14/14   
Abnormal esophagus physiology [HP:0025270]            8/8    100%  16/16   
Autistic behavior [HP:0000729]                       8/17     47%  14/27   
Diagnostic behavioral phenotype [HP:0025783]          8/8    100%  14/14   
Atypical behavior [HP:0000708]                      10/10    100%  16/16   

Sex of the individual                                                          \
                                                   Percent Corrected p values   
Hypothyroidism [HP:0000821]                            45%           0.593472   
Feeding difficulties [HP:0011968]                     100%           1.000000   
Gastroesophageal reflux [HP